In [1]:
####################################################
#File name:IdentificationOfWorkLocation_rev4
#Creation date:2020/9/30
#Updating date:2020/10/20
#Shizuka Hamajima
# rev4はテストデータを読み込んでファイルに結果出力させる。
# 精度検証に用いるためのプログラム
####################################################
import pandas as pd
from pandas import Series, DataFrame

#正規表現操作のインポート
import re

In [2]:
cd ./ThresholdData/

C:\Users\01032214\Programming\2020Sep_IdentificationOfWorkLocation\ThresholdData


In [3]:
#最低賃金表の読み込み
minimum_wage_list = pd.read_csv('minimum_wage.csv',encoding="SHIFT-JIS")

In [4]:
#最低賃金表読み込み結果
minimum_wage_list

,Class,Lower_limit,Upper_limit,Minimum_wage,Regular_Expressions
0,1,790,799,798,秋田|鳥取|島根|高知|佐賀|大分|沖縄|青森|岩手|山形|愛媛|長崎|熊本|宮崎|鹿児島|...
1,2,810,819,818,四国|香川
2,3,820,829,829,東北|宮城|山口|福井
3,4,830,839,837,新潟|和歌山|石川|岡山|群馬|山梨|奈良
4,5,840,849,849,九州|福岡|北九州|富山|長野|茨城
5,6,850,859,853,岐阜|栃木
6,7,860,869,866,北海道|札幌|滋賀
7,8,870,879,873,中国|広島|三重
8,9,880,889,885,静岡
9,10,890,899,899,兵庫|神戸


In [5]:
cd ..

C:\Users\01032214\Programming\2020Sep_IdentificationOfWorkLocation


In [6]:
cd ./TestData/

C:\Users\01032214\Programming\2020Sep_IdentificationOfWorkLocation\TestData


In [7]:
#チェック対象テキストデータの読み込み
check_data = pd.read_csv('WorkLocationTest_Input.csv',encoding="SHIFT-JIS")
#check_data

In [8]:
#勤務地初期値設定
check_data['WorkLocation'] = "不明"
#最低賃金の閾値初期化
check_data['MinimumWage'] = minimum_wage_list.Minimum_wage.max()
check_data.head()

,scd,title,text,WorkLocation,MinimumWage
0,97633001,勤務地,【首都圏限定の店舗展開】\n\n\n\n※各営業店舗14店舗\n\n【東京都】 新宿、池袋、...,不明,1013
1,260555001,勤務地（転勤はありません）,鳥取県内の各店舗（自宅通勤可能な店舗への配属）\n\n・米子店／米子市東福原\n\n・倉吉店...,不明,1013
2,1636789004,勤務地,東京都内、神奈川県、埼玉県内の各校舎63校及び5つの地区オフィス\n\n◆東京都\n\n23...,不明,1013
3,2346829004,勤務地,■法人固定回線事業\n\n福岡県久留米市宮ノ陣4-29-11\n\n東京都中央区日本橋本町3...,不明,1013
4,85965001,勤務地,当社事業所,不明,1013


In [9]:
#勤務地クラス数　改訂時に可変にするため
N = minimum_wage_list.Class.max()
#print(N)  #確認
###勤務地判定
#勤務地判定用正規表現初期化
pattern = [""] * N
#print(pattern)  #確認
for i in range(N):
    pattern[i] = minimum_wage_list.loc[i,"Regular_Expressions"]
#print(pattern)  #確認
#最賃昇順に勤務地判定
for j in range(len(check_data)):
    for i in range(N):
        work_location_text = re.findall(pattern[i], check_data.loc[j,"text"])
        if len(work_location_text) != 0:
            check_data.loc[j,"WorkLocation"] = minimum_wage_list.loc[i,"Class"]
            check_data.loc[j,"MinimumWage"] = minimum_wage_list.loc[i,"Minimum_wage"]

In [10]:
#勤務地判定結果ファイル出力
check_data.to_csv("WorkLocationTest_Output.csv",encoding="SHIFT-JIS")
check_data.head()

,scd,title,text,WorkLocation,MinimumWage
0,97633001,勤務地,【首都圏限定の店舗展開】\n\n\n\n※各営業店舗14店舗\n\n【東京都】 新宿、池袋、...,14,1013
1,260555001,勤務地（転勤はありません）,鳥取県内の各店舗（自宅通勤可能な店舗への配属）\n\n・米子店／米子市東福原\n\n・倉吉店...,1,798
2,1636789004,勤務地,東京都内、神奈川県、埼玉県内の各校舎63校及び5つの地区オフィス\n\n◆東京都\n\n23...,14,1013
3,2346829004,勤務地,■法人固定回線事業\n\n福岡県久留米市宮ノ陣4-29-11\n\n東京都中央区日本橋本町3...,14,1013
4,85965001,勤務地,当社事業所,不明,1013
